# Big data made small

Follow along the notebook to learn how to use SQL through [DuckDB](https://DuckDB.org/), a **fast** database system, available through both python and R.

In [1]:
# Import today's main character
import duckdb

## Basic building blocks of SQL querying

SQL is a declarative language that allows you to query tables from databases. There's plenty of database ecosystems out there. If you work with data in a large organization, you'll encounter at least one. Often more than one.

DuckDB is an **analytical** database system that allows extreme flexibility in querying data in different formats. Today we will focus primarily on `parquet` files (though you could also directly work on e.g. `CSV`s).

[`parquet`](https://parquet.apache.org/) is a data format optimized for efficient data retrieval. There's more and more data available as parquet files out there, and there's multiple way to transform your data into parquet - the easiest probably being [exporting from pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html). 

For today, we'll assume you got the data in this format, which will be created in your local `./data/` folder as soon as you run `uv run create_data.py`.

This introduction runs through the basic syntax of `SELECT` statements in SQL. These constitutes the vast majority of SQL work for an analyst.

### SELECT FROM + LIMIT

`SELECT` statements are structured as follows

![image](../imgs/select.png)

A basic query could then look something like

```SQL
SELECT
    column1, column2
FROM thistable
LIMIT 5
```

This query means "Extract the first 5 records of columns `column1` and `column2` from table `thistable`.

You can also usethe character `*` as jolly to return all columns in a table.

> **WARNING**
> 
> If you are doing exploratory analysis "just to look at the data", **always** impose a `LIMIT` condition to ensure you don't load everything in the memory. DuckDB (and database engines in general) can handle larger-than-RAM datasets. Loading a huge table  up is a safe recipe for crashing your computer sooner or later.

DuckDB allows to run SQL queries **directly on your `parquet` files**! This means you will not load the whole dataset in memory. DuckDB will intelligently scan the data, select only the tidbits you need, alaborate them, and return them.

So say we want to "look at what's in `../data/tests.parquet` file. We can adapt the basic query above and transform it into  

```SQL
select te.*
from read_parquet('../data/tests.parquet') as te
limit 5
```

Where `read_parquet()` is a DuckDB function reading parquet files.

> **NOTE**
>
> Strings in SQL always are delimited by character `'`, **not** `"`


With DuckDB, we can run SQL query simply by encapsulating them into `duckdb.sql()` as follows 

In [2]:
%%time
duckdb.sql(
    """
    select te.*
    from read_parquet('../data/tests.parquet') as te
    limit 5
    """
)

CPU times: user 0 ns, sys: 1.73 ms, total: 1.73 ms
Wall time: 1.41 ms


┌─────────────────────┬───────┬──────────┐
│      test_time      │  id   │ positive │
│    timestamp_ns     │ int64 │  int64   │
├─────────────────────┼───────┼──────────┤
│ 2022-01-08 12:56:02 │     1 │        0 │
│ 2022-02-19 05:04:32 │     1 │        0 │
│ 2022-02-20 14:51:55 │     1 │        0 │
│ 2022-02-21 15:14:18 │     1 │        0 │
│ 2022-05-10 16:21:15 │     1 │        0 │
└─────────────────────┴───────┴──────────┘

Easy! We had a peek at the data in a tiny fraction of a second, without the need of loading all the dataset in the memory.

`duckdb.sql()` returns a relation object. You can convert it in other formats, like a `pandas.DataFrame`, e.g.

In [3]:
%%time
df = duckdb.sql(
    """
    select te.*
    from read_parquet('../data/tests.parquet') as te
    limit 5
    """
).df()

df

CPU times: user 752 ms, sys: 480 ms, total: 1.23 s
Wall time: 304 ms


,test_time,id,positive
0,2022-01-08 12:56:02,1,0
1,2022-02-19 05:04:32,1,0
2,2022-02-20 14:51:55,1,0
3,2022-02-21 15:14:18,1,0
4,2022-05-10 16:21:15,1,0


### WHERE and ORDER BY

We can add a condition to the query. For example, we can get all tests done by `id=5` after August 2022 by using a `WHERE` statement.

Note that now I am not using a `LIMIT` condition anymore, as I know the data output will be limited in this case.

I am also using `ORDER BY` at the end to order rows chronologically.

In [4]:
%%time
duckdb.sql(
    """
    select te.*
    from read_parquet('../data/tests.parquet') as te
    where te.id=5
        and cast(te.test_time as date)>='2022-08-01'
    ORDER BY te.test_time
    """
)

CPU times: user 1.16 ms, sys: 0 ns, total: 1.16 ms
Wall time: 710 µs


┌─────────────────────┬───────┬──────────┐
│      test_time      │  id   │ positive │
│    timestamp_ns     │ int64 │  int64   │
├─────────────────────┼───────┼──────────┤
│ 2022-08-13 01:20:47 │     5 │        0 │
│ 2022-08-31 09:26:49 │     5 │        0 │
│ 2022-09-10 07:38:21 │     5 │        0 │
│ 2022-11-12 11:50:07 │     5 │        0 │
└─────────────────────┴───────┴──────────┘

### GROUP BY

`GROUP BY` serves to aggregate our data. Via SQL, you can aggregate the data *without* loading all rows in a pandas dataframe. For example, say we want to count how many tests have been done by month in the `tests` data, and the percentage of them showing up positive (here data is simulated, so no useful pattern will show up).

Here's how you could do it.

In [5]:
%%time
duckdb.sql(
    """
    select
        month(test_time) as month
        , COUNT(*) as n_tests
        , MEAN(positive) as share_positive
    from read_parquet('../data/tests.parquet') as te
    GROUP BY month(test_time)
    ORDER BY month(test_time)
    """
)

CPU times: user 2.36 ms, sys: 1.15 ms, total: 3.52 ms
Wall time: 2.08 ms


┌───────┬─────────┬─────────────────────┐
│ month │ n_tests │   share_positive    │
│ int64 │  int64  │       double        │
├───────┼─────────┼─────────────────────┤
│     1 │ 3834859 │ 0.07970618997986627 │
│     2 │ 3459620 │ 0.07996514068019031 │
│     3 │ 3829567 │ 0.07966592567775939 │
│     4 │ 3710265 │ 0.07995116251804116 │
│     5 │ 3835185 │ 0.07991817865370249 │
│     6 │ 3705759 │ 0.08002328267974253 │
│     7 │ 3833010 │ 0.07986386677832825 │
│     8 │ 3830474 │ 0.08007520740253034 │
│     9 │ 3708918 │ 0.07996024716642428 │
│    10 │ 3835433 │  0.0799581168540814 │
│    11 │ 3708389 │ 0.07998702401501029 │
│    12 │ 3708521 │ 0.08023252396305697 │
├───────┴─────────┴─────────────────────┤
│ 12 rows                     3 columns │
└───────────────────────────────────────┘

## Expanding `FROM`

### Subqueries and Common Table Expressions (CTEs)

Let's go back to `FROM`. The easiest way of using `FROM` is to use as input a single table. But a table can also be the result of a query itself!

For example, let's see how the distribution of many times each person in the dataset was tested. To do so, we need to

1. calculate the number of tests per id in the data
2. sum how many ID occurrences per number of tests. 

Or:

In [6]:
%%time
duckdb.sql(
    """
    SELECT n_tests, count(id) as n_ppl 
    FROM (
        SELECT id, count(id) as n_tests 
        from read_parquet('../data/tests.parquet') 
        GROUP BY id
    ) as t
    GROUP BY n_tests
    ORDER BY n_tests
    """
)

CPU times: user 1.94 ms, sys: 420 µs, total: 2.36 ms
Wall time: 1.17 ms


┌─────────┬────────┐
│ n_tests │ n_ppl  │
│  int64  │ int64  │
├─────────┼────────┤
│       1 │     12 │
│       2 │    101 │
│       3 │    515 │
│       4 │   1927 │
│       5 │   5826 │
│       6 │  14424 │
│       7 │  30894 │
│       8 │  57953 │
│       9 │  97722 │
│      10 │ 145706 │
│       · │     ·  │
│       · │     ·  │
│       · │     ·  │
│      29 │   1240 │
│      30 │    680 │
│      31 │    359 │
│      32 │    136 │
│      33 │     57 │
│      34 │     25 │
│      35 │     11 │
│      36 │      3 │
│      37 │      1 │
│      39 │      1 │
├─────────┴────────┤
│     38 rows      │
│    (20 shown)    │
└──────────────────┘

The part in parentheses after `FROM` is a subquery. The problem with subqueries is that, even with copious indentation, queries quickly become hard to read and debug.

Enter **Common Table Expressions (CTEs)**. They are almost the same as subqueries, but we defined them *at the beginning of a query*, encapsulating them into a `WITH ctename as (CTE)` statement, making the query gets more legible.

Another advantage of CTEs is that we'd be able to reuse table `t` in the example below multiple times in a query, if needed, without rewriting the subquery multiple times. As such, CTEs serve as temporary tables.

In [7]:
%%time
duckdb.sql(
    """
    WITH t as (
        SELECT id, count(id) as n_tests 
        from read_parquet('../data/tests.parquet') 
        GROUP BY id
    )
    SELECT n_tests, count(id) as n_ppl 
    FROM t
    GROUP BY n_tests
    ORDER BY n_tests
    """
)

CPU times: user 1.61 ms, sys: 222 µs, total: 1.83 ms
Wall time: 843 µs


┌─────────┬────────┐
│ n_tests │ n_ppl  │
│  int64  │ int64  │
├─────────┼────────┤
│       1 │     12 │
│       2 │    101 │
│       3 │    515 │
│       4 │   1927 │
│       5 │   5826 │
│       6 │  14424 │
│       7 │  30894 │
│       8 │  57953 │
│       9 │  97722 │
│      10 │ 145706 │
│       · │     ·  │
│       · │     ·  │
│       · │     ·  │
│      29 │   1240 │
│      30 │    680 │
│      31 │    359 │
│      32 │    136 │
│      33 │     57 │
│      34 │     25 │
│      35 │     11 │
│      36 │      3 │
│      37 │      1 │
│      39 │      1 │
├─────────┴────────┤
│     38 rows      │
│    (20 shown)    │
└──────────────────┘

### The real deal: JOINs

As mentioned in the slides, working with relational databases means joining lots of table together. That's the real superpower of SQL.

`JOIN` is how we combine multiple tables. Essentially that'd be `pd.DataFrame.merge()` - just exceptionally more powerful.

There are different types of `JOIN`s. The table below summarizes the key differences, and I will explain them briefly at the workshop, but as analysts you'd most often deal with two of these: `INNER JOIN` and `LEFT JOIN`.

![image](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.techagilist.com%2Fwp-content%2Fuploads%2F2018%2F07%2Fsql-joins.png&f=1&nofb=1&ipt=187475e01030755dedb9dcd23da61bdb8650b747c4fca8004eb442d243191213&ipo=images)

The idea is simply to take all rows from a table, and attach all rows from a second table according to a (set of) column(s). So for example I want to join each id from Table A to each ID of Table B.

* `INNER JOIN`: Returns *only rows where IDs are found in **both** tables*
* `LEFT JOIN`: Returns *all rows from table A*, joining data from Table B when available (the rest being `NULL`)

In the simulated data, for example, let's say that the `tests` data contains tests for all the population, while the `transactions` data only contains transaction of clients of a specific financial institute. Indeed, we could count how many `DISTINCT` IDs we have in both tables.

In [8]:
%%time
print(
    duckdb.sql(
        """select count(distinct id) as id_tests
        from read_parquet('../data/tests.parquet')
        """
    )
)
print(
    duckdb.sql(
        """
        select count(distinct id) as id_transactions
        from read_parquet('../data/transactions.parquet')
        """
    )
)

┌──────────┐
│ id_tests │
│  int64   │
├──────────┤
│  2999999 │
└──────────┘

┌─────────────────┐
│ id_transactions │
│      int64      │
├─────────────────┤
│          899999 │
└─────────────────┘

CPU times: user 4.26 s, sys: 20.4 ms, total: 4.28 s
Wall time: 397 ms


Let's say we want to create a dataframe with as columns:
- The ID of the person (key)
- Whether a person ever tasted positive
- The number of transactions the person made

To do so, first we aggregate the tests and transaction table, then we join them.

Let's start with a `LEFT JOIN`, where the `LEFT` table is the aggregated test table.

We get out 2999999 rows (the number of unique IDs in the `tests` table), but many of them have a missing `n_transactions`. We do not have transaction data for those individuals. However, we still retain the full sample, as we **do** have test data for the full population.

In [9]:
%%time
duckdb.sql(
    """
    with te as (
        SELECT id, max(positive) as ever_positive
        FROM read_parquet('../data/tests.parquet')
        GROUP BY id
    ), tr as (
        SELECT id, count(id) as n_transactions
        FROM read_parquet('../data/transactions.parquet')
        GROUP BY id
    )
    SELECT
        te.*, tr.n_transactions
    FROM te
        LEFT JOIN tr
            ON te.id=tr.id
    ORDER BY te.id DESC
    """
).df()

CPU times: user 6.15 s, sys: 295 ms, total: 6.44 s
Wall time: 738 ms


,id,ever_positive,n_transactions
0,2999999,1,NaN
1,2999998,1,NaN
2,2999997,1,NaN
3,2999996,1,NaN
4,2999995,1,NaN
...,...,...,...
2999994,5,0,121.0
2999995,4,1,113.0
2999996,3,1,113.0
2999997,2,1,92.0


Were we to do an `INNER JOIN` instead, we'd get 899999 rows - the clients having *both* test and transaction data.

In [10]:
%%time
duckdb.sql(
    """
    with te as (
        SELECT id, max(positive) as ever_positive
        FROM read_parquet('../data/tests.parquet')
        GROUP BY id
    ), tr as (
        SELECT id, count(id) as n_transactions
        FROM read_parquet('../data/transactions.parquet')
        GROUP BY id
    )
    SELECT
        count(*)
    FROM te
    INNER JOIN tr
        ON te.id=tr.id
    """
)

CPU times: user 3.14 ms, sys: 182 µs, total: 3.32 ms
Wall time: 1.82 ms


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       899999 │
└──────────────┘

### Non-trivial JOINs

The joins we have seen above are quite straightfoward: One key, with equality. These joins are also straighforward with e.g. pandas. However, SQL allows you to handle much more complex joins.

For example, joins on 
- Multiple keys (e.g. id and date)
- Functions of keys (e.g. month of a date)
- Unequal joins (e.g. on X>Y)

To try these out, let's try to find out all transactions made by people who tested positive 2 to 5 days before the positive test: That is, when they were asymptomatic but likely contagious.

Now, this starts to be a somewhat complex operation. We need to:

* Extract all positive tests. That's 3.597.337 positive tests in the simulated data.
* Find all transactions made by the individual being tested. The number of rows would grow to 108.989.386 transactions.
* Only select those where the time range fits.

The code belows does this join in `pandas`. It's uncommented, as you should only try it out if you have ~10GB RAM memory to spare. It takes about 35 seconds to run on my machine.

> **Note:** There'll be a few records not matching due to how pandas and sql engines treat date boundaries.

In [11]:
# %%time
# import pandas as pd
# tests = pd.read_parquet('../data/tests.parquet')
# transactions = pd.read_parquet('../data/transactions.parquet')

# pt = tests.loc[tests["positive"]==1]
# tt = pt.merge(transactions, on='id', how='inner')
# tt["infector_days_to_test"] = (tt["transaction_time"] - tt["test_time"]).dt.days + 1
# tt = tt[tt["infector_days_to_test"].between(-5, -2)]
# tt

The same query in in DuckDB runs in ~2 seconds on my machine, and the RAM is not nearly as challenged.

In [12]:
%%time
experiments = duckdb.sql(
    """
    select t.id as infector_id, t.test_time as infector_test_time
        , tt.transaction_time as infector_transaction_time, tt.shop
        , date_sub('days', t.test_time, tt.transaction_time) as infector_days_to_test
        --, row_number() over() as experiment_number
    from read_parquet('../data/tests.parquet') as t
    inner join read_parquet('../data/transactions.parquet') as tt
        on tt.id=t.id 
        and date_sub('days', t.test_time, tt.transaction_time) between -5 and -2
    where t.positive=1
    --order by t.id, infector_test_date, infector_transaction_time
    """
).df()
experiments

CPU times: user 13.7 s, sys: 392 ms, total: 14.1 s
Wall time: 1.35 s


,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test
0,2,2022-12-23 23:15:26,2022-12-18 04:22:22,47110,-5
1,3,2022-05-23 04:49:18,2022-05-17 15:36:40,9413,-5
2,3,2022-05-23 04:49:18,2022-05-18 15:55:45,14896,-4
3,3,2022-05-23 04:49:18,2022-05-21 03:22:43,30801,-2
4,6,2022-06-24 00:24:28,2022-06-20 07:46:20,7967,-3
...,...,...,...,...,...
1184794,889396,2022-09-23 23:05:12,2022-09-21 15:59:43,27639,-2
1184795,889396,2022-08-17 00:01:23,2022-08-11 05:41:44,26702,-5
1184796,889396,2022-06-07 11:09:38,2022-06-01 13:32:00,23059,-5
1184797,889396,2022-06-07 11:09:38,2022-06-03 19:43:05,42277,-3


## All together: Find all transactions by people crossing an infector in a store!

Now the final step. Let's take the *experiments* we identified in the previous query, and join them with all transactions by other people happening in the same store within 30 minutes of the original transactions. Piece of cake!

>The way I'd approach building the query:
>1) Start from the basic ingredients, i.e. the tables. We have a query creating our "experiments". Start from that.
>2) Understand which data I need additionally - here the `transactions` data. Let's `JOIN` it.
>3) Write down the conditions under which combinations of transactions are acceptable. For example, they need to be
>    * In the same shop
>    * Not by the same person (the "*infector*")
>    * Within a specific time range
>4) Potentially construct additional variables (here "treatment")

In [13]:
%%time
duckdb.sql(
    """
    with experiments as (
        select t.id as infector_id, t.test_time as infector_test_time
            , tt.transaction_time as infector_transaction_time, tt.shop
            , date_sub('days', t.test_time, tt.transaction_time) as infector_days_to_test
            , row_number() over() as experiment_number
        from read_parquet('../data/tests.parquet') as t
        inner join read_parquet('../data/transactions.parquet') as tt
            on tt.id=t.id 
            and date_sub('days', t.test_time, tt.transaction_time) between -5 and -2
        where t.positive=1
        -- a small number to test it out
        limit 1000
    ) 
    select e.*
    , t2.id as exposed_id, t2.transaction_time as exposed_transaction_time
    , date_sub('minutes', t2.transaction_time, e.infector_transaction_time) as minutes_apart
    , case when date_sub('minutes', t2.transaction_time, e.infector_transaction_time) between -5 and +5 then 1 
        when date_sub('minutes', t2.transaction_time, e.infector_transaction_time) between -30 and -16 then 0
        else NULL end as treat
    from experiments as e
    inner join read_parquet('../data/transactions.parquet') as t2
        -- in the same shop
        on t2.shop=e.shop
        -- I do not want the infector transactions there!
        and t2.id!=e.infector_id
        --order by experiment_number, exposed_transaction_time
        and t2.transaction_time between
            e.infector_transaction_time - interval 30 minutes
            and e.infector_transaction_time + interval 30 minutes
    """
).df()

CPU times: user 7.84 s, sys: 349 ms, total: 8.19 s
Wall time: 761 ms


,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test,experiment_number,exposed_id,exposed_transaction_time,minutes_apart,treat
0,688,2022-06-11 22:07:12,2022-06-08 05:14:58,13643,-3,935,115443,2022-06-08 05:23:52,-8,NaN
1,299,2022-06-04 01:16:47,2022-05-29 06:16:11,40698,-5,447,168471,2022-05-29 06:44:04,-27,0.0
2,536,2022-01-08 06:45:45,2022-01-03 02:04:06,31460,-5,764,170889,2022-01-03 01:44:20,19,NaN
3,463,2022-06-28 09:15:08,2022-06-25 14:55:33,11760,-2,654,173642,2022-06-25 14:36:06,19,NaN
4,126,2022-06-06 09:35:17,2022-05-31 16:51:12,1618,-5,200,314288,2022-05-31 17:13:40,-22,0.0
...,...,...,...,...,...,...,...,...,...,...
206,514,2022-09-11 11:39:57,2022-09-05 18:07:02,47551,-5,734,756854,2022-09-05 18:20:59,-13,NaN
207,441,2022-03-04 01:42:10,2022-02-28 03:51:23,22724,-3,638,759153,2022-02-28 03:36:11,15,NaN
208,644,2022-01-28 16:54:19,2022-01-24 05:34:48,39845,-4,886,760117,2022-01-24 05:05:33,29,NaN
209,372,2022-07-20 16:54:30,2022-07-15 19:44:38,36589,-4,561,760230,2022-07-15 19:54:51,-10,NaN


### Using dataframes in memory in your queries

One of the blackest of black magics of DuckDB when you are using it through a python API is that you can directly refer in the queries to existing `pandas.DataFrame`s in memory.

For example, in the code above we saved the results of our `experiments` query in a specific dataframe.

In [14]:
experiments.head()

,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test
0,2,2022-12-23 23:15:26,2022-12-18 04:22:22,47110,-5
1,3,2022-05-23 04:49:18,2022-05-17 15:36:40,9413,-5
2,3,2022-05-23 04:49:18,2022-05-18 15:55:45,14896,-4
3,3,2022-05-23 04:49:18,2022-05-21 03:22:43,30801,-2
4,6,2022-06-24 00:24:28,2022-06-20 07:46:20,7967,-3


Here we can just reference it in the query below. There is no difference in perfomance here - but it can be an extremely useful tool when working with data coming from different sources.

In [15]:
%%time
duckdb.sql(
    """
    select e.*
    , t2.id as exposed_id, t2.transaction_time as exposed_transaction_time
    , date_sub('minutes', t2.transaction_time, e.infector_transaction_time) as minutes_apart
    , case when date_sub('minutes', t2.transaction_time, e.infector_transaction_time) between -5 and +5 then 1 
        when date_sub('minutes', t2.transaction_time, e.infector_transaction_time) between -30 and -16 then 0
        else NULL end as treat
    from experiments as e
    inner join read_parquet('../data/transactions.parquet') as t2
        -- in the same shop
        on t2.shop=e.shop
        -- I do not want the infector transactions there!
        and t2.id!=e.infector_id
        --order by experiment_number, exposed_transaction_time
        and t2.transaction_time between
            e.infector_transaction_time - interval 30 minutes
            and e.infector_transaction_time + interval 30 minutes
    limit 100
    """
).df()

CPU times: user 25.4 s, sys: 141 ms, total: 25.5 s
Wall time: 2.74 s


,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test,exposed_id,exposed_transaction_time,minutes_apart,treat
0,553184,2022-11-27 16:24:18,2022-11-21 20:08:03,17982,-5,6,2022-11-21 20:32:18,-24,0.0
1,511684,2022-11-24 09:46:30,2022-11-21 20:06:11,17982,-2,6,2022-11-21 20:32:18,-26,0.0
2,551104,2022-02-12 01:20:31,2022-02-08 05:06:17,35412,-3,19,2022-02-08 05:19:47,-13,NaN
3,199969,2022-04-19 23:24:36,2022-04-15 04:42:23,11364,-4,9,2022-04-15 04:21:36,20,NaN
4,23105,2022-07-07 04:50:41,2022-07-02 19:30:43,28537,-4,6,2022-07-02 19:57:08,-26,0.0
...,...,...,...,...,...,...,...,...,...
95,820964,2022-07-22 15:03:31,2022-07-17 23:26:13,35779,-4,264,2022-07-17 23:06:31,19,NaN
96,676475,2022-01-18 18:11:33,2022-01-16 16:54:37,6144,-2,291,2022-01-16 16:39:40,14,NaN
97,877649,2022-04-09 20:09:08,2022-04-07 17:02:26,9417,-2,296,2022-04-07 16:56:59,5,1.0
98,459349,2022-04-03 00:54:58,2022-03-28 11:00:00,18546,-5,293,2022-03-28 10:31:50,28,NaN


### Data versioning becoming code versioning

Putting your queries in notebooks is all fine and well for quick analysis. However, among their many problems, notebooks famously do not agree much with proper code versioning.

However, we could easily save our queries in separate files, and refer to the in our analytical products. That way, the **queries** themselves can be versioned. And by properly using version control, it allows you to essentially version control your data if your raw data sources are immutable.

For fun, the example below shows you also how to parametrize your query.

In [16]:
def read_file(filename):
    try:
        with open(filename, 'r') as file:
            content = file.read()
            return content
    except FileNotFoundError:
        print(f"File {filename} not found.")
        return None

duckdb.execute(read_file("myquery.sql"), parameters={'nlimit': 100}).df()

,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test,experiment_number,exposed_id,exposed_transaction_time,minutes_apart,treat
0,28,2022-04-06 12:19:45,2022-04-01 21:48:10,918,-4,35,529958,2022-04-01 21:20:47,27,NaN
1,53,2022-03-20 08:21:46,2022-03-17 06:48:44,22037,-3,71,455829,2022-03-17 06:19:27,29,NaN
2,4,2022-06-28 18:17:52,2022-06-23 20:07:24,21100,-4,16,26078,2022-06-23 20:34:48,-27,0.0
3,60,2022-09-20 07:23:59,2022-09-17 11:12:23,116,-2,83,316799,2022-09-17 11:14:15,-1,1.0
4,3,2022-05-23 04:49:18,2022-05-17 15:36:40,9413,-5,2,145252,2022-05-17 15:15:36,21,NaN
5,9,2022-08-01 13:08:09,2022-07-28 15:51:27,27737,-3,6,222151,2022-07-28 16:00:24,-8,NaN
6,46,2022-08-22 15:01:30,2022-08-20 09:36:55,47846,-2,91,345563,2022-08-20 09:49:38,-12,NaN
7,55,2022-09-20 09:07:03,2022-09-16 22:07:16,34410,-3,72,696629,2022-09-16 22:28:32,-21,0.0
8,8,2022-06-15 04:15:55,2022-06-11 01:47:12,30665,-4,18,71808,2022-06-11 01:42:57,4,1.0
9,43,2022-06-29 20:03:43,2022-06-26 14:37:44,33929,-3,63,480760,2022-06-26 14:45:12,-7,NaN
